In [1]:
import pandas as pd
import requests, os
from time import sleep

In [2]:
# ## Apurar lista generalizada de produtos
produtos = list()
data = {'formato': 'json',
        'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
        'pesquisa': '',
        'pagina': 1}
while True:
    session = requests.post(url='https://api.tiny.com.br/api2/produtos.pesquisa.php', data=data).json()
    data['pagina'] += 1
    try:
        produtos += [x['produto'] for x in session['retorno']['produtos']]
    except:
        break
produtos = pd.DataFrame(produtos)


In [36]:

# Apurar databaset completo de produtos
produtos_completos = list()
for id in produtos['id']:
    data = {
        'formato': 'json',
        'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
        'id': id
    }
    try:
        produtos_completos += [requests.post(url='https://api.tiny.com.br/api2/produto.obter.php', data=data).json()['retorno']['produto']]
    except:
        print('limite de requisições')
        sleep(60)
        produtos_completos += [requests.post(url='https://api.tiny.com.br/api2/produto.obter.php', data=data).json()['retorno']['produto']]
produtos_completos = pd.DataFrame(produtos_completos)


limite de requisições
limite de requisições


In [37]:

# Filtragem de dados
produtos_filtrados = produtos_completos[pd.DataFrame([produtos_completos['categoria'] == x for x in ['Bovinos', 'Suínos', 'Aves', 'Especiais']]).any()]
produtos_filtrados = produtos_filtrados[produtos_filtrados['slug'] != '']


In [38]:

# Limpeza de dados
produtos_formatados = produtos_filtrados.reindex(['id', 'nome', 'codigo', 'unidade', 'estoque_minimo', 'slug', 'marca'], axis=1)
produtos_formatados['unidade'] = produtos_formatados['unidade'].str.casefold()
produtos_formatados['unidade'] = ['un' if x != 'kg' else 'kg' for x in produtos_formatados['unidade']]
produtos_formatados


,id,nome,codigo,unidade,estoque_minimo,slug,marca
1,623449734,ALCATRA COM QUEIJO,ALCAQP,kg,8.62,Alcatra,Porcão
2,623449795,ALMÔNDEGA BACON,ALMOBARB,kg,5.00,Almôndega,Recheados Brasil
3,623449782,ALMÔNDEGA QUEIJO,ALMOQRB,kg,4.50,Almôndega,Recheados Brasil
4,778208483,ANCHO ALIMENTAR,ANCHALI,kg,0.00,Ancho,Alimentar
5,646749750,ANCHO DI PRIMA,ANCHDIPDI,kg,0.00,Ancho,Di Prima
...,...,...,...,...,...,...,...
143,638867885,T-BONE MINERVA ANGUS,TBONMINE,kg,5.00,T-Bone,Angus
144,640758272,T-BONE RARO,TBNERARO,kg,0.00,T-Bone,Raro
146,623450791,TOMAHAWK CARA PRETA,TOMACP,kg,2.00,Tomahawk,Cara Preta
147,647280670,TOMAHAWK RARO,TOMARARO,kg,0.00,Tomahawk,Raro


In [39]:

# POST /produtos
produtos_estoque = produtos_formatados
estoque = list()
for id in produtos_estoque['id']:
    data = {
        'formato': 'json',
        'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
        'id': id
    }
    try:
        estoque += [requests.post(url='https://api.tiny.com.br/api2/produto.obter.estoque.php', data=data).json()['retorno']['produto']['saldo']]
    except:
        print('limite de requisições')
        sleep(60)
        estoque += [requests.post(url='https://api.tiny.com.br/api2/produto.obter.estoque.php', data=data).json()['retorno']['produto']]
produtos_estoque['estoque'] = estoque

limite de requisições
limite de requisições


In [41]:
produtos_estoque[produtos_estoque['marca'] == '']

,id,nome,codigo,unidade,estoque_minimo,slug,marca,estoque
10,661843960,BACON FATIADO,BACNFATDNT,un,2.0,Bacon,,0
48,789712012,HAMBURGUER DE TILÁPIA,HBRGTILP,un,0.0,Hamburguer,,16
58,785293008,LINGUIÇA ARTESANAL DE TILÁPIA,LINGARTTLP,kg,0.0,Linguiça,,14.5
69,804981063,LINGUIÇA DE PERNIL GARAPA,LINGPERNGARA,kg,0.0,Linguiça,,0
72,653960156,MAMINHA STEAK MARINADO,MAMSTKPORC,kg,4.0,Maminha,,0
88,804677691,PÃO DE FRANGO COM REQUEIJAO,PDFDNBT,un,0.0,Pão de frango,,0
121,687343884,PICOLÉ MINEIRO BBQ,PCLMINRBBBQ,un,4.0,Picolé Mineiro,,0
122,687345363,PICOLÉ MINEIRO ERVAS FINAS,PCLMINRBEF,un,4.0,Picolé Mineiro,,13
141,692082492,STEAK DE LOMBO,STKLOMBDIA,kg,2.0,Steak,,0
